In [ ]:
#==강사님 번호판 코드

In [2]:
# Jupyter Notebook 셀에 그대로 실행
import os, io, re, shutil
from pathlib import Path
from PIL import Image, UnidentifiedImageError
import ipywidgets as widgets
from IPython.display import display

# ===== 사용자 설정 =====
# DEFAULT_DIR = r"C:/Users/Admin/single_line"   # 바꾸세요
DEFAULT_DIR = r"D:/workspace/data1000"   # 바꾸세요
IMAGE_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".gif", ".webp", ".tif", ".tiff"}

THUMB_MAX_SIDE = 512          # 썸네일 최대 변
THUMB_DISPLAY_WIDTH = 280     # 썸네일 표시 폭
ZOOM_MAX_SIDE = 1280          # Zoom 최대 변
# PAGE_SIZE = 50                # ★ 한 페이지 50개
PAGE_SIZE = 30
ALLOW_OVERWRITE = False

# ===== Windows 파일명 유효성 검사 =====
_reserved = {
    "CON","PRN","AUX","NUL","COM1","COM2","COM3","COM4","COM5","COM6","COM7","COM8","COM9",
    "LPT1","LPT2","LPT3","LPT4","LPT5","LPT6","LPT7","LPT8","LPT9"
}
_invalid_re = re.compile(r'[\\/:*?"<>|]')
def validate_windows_filename(stem:str):
    s = stem.strip()
    if not s: return False, "빈 파일명은 허용되지 않습니다."
    if s.upper() in _reserved: return False, f"예약어({s})는 파일명으로 사용할 수 없습니다."
    if _invalid_re.search(s): return False, r'파일명에 \ / : * ? " < > | 문자를 사용할 수 없습니다.'
    if s.endswith(".") or s.endswith(" "): return False, "파일명은 점(.)이나 공백으로 끝날 수 없습니다."
    return True, ""

# ===== 이미지 바이트 =====
def _to_bytes(path: Path, max_side:int):
    try:
        with Image.open(path) as im:
            im = im.convert("RGB")
            im.thumbnail((max_side, max_side))
            bio = io.BytesIO()
            im.save(bio, format="PNG", optimize=True)
            return bio.getvalue()
    except UnidentifiedImageError:
        return None

def make_thumb_bytes(path: Path): return _to_bytes(path, THUMB_MAX_SIDE)
def make_zoom_bytes(path: Path):  return _to_bytes(path, ZOOM_MAX_SIDE)

# ===== 상단 UI =====
dir_text = widgets.Text(value=DEFAULT_DIR, description="Folder:", layout=widgets.Layout(width="70%"))
load_btn = widgets.Button(description="Load", button_style="primary")
rename_all_btn = widgets.Button(description="Rename All")
overwrite_chk = widgets.Checkbox(value=ALLOW_OVERWRITE, description="Overwrite if exists")

# 페이지 네비
pager_box_top = widgets.HBox()
pager_box_bottom = widgets.HBox()

# 목록 / 상태
items_box = widgets.VBox()
status_out = widgets.Output(layout={"border": "1px solid #ddd"})

display(widgets.HBox([dir_text, load_btn, rename_all_btn, overwrite_chk]),
        pager_box_top, items_box, pager_box_bottom, status_out)

# ===== 상태 =====
_file_rows = []     # 현재 페이지 행들
_all_files = []     # 전체 파일 목록
_curr_page = 1
_total_pages = 1

def list_images(folder: Path):
    return sorted([p for p in folder.iterdir() if p.is_file() and p.suffix.lower() in IMAGE_EXTS],
                  key=lambda p: p.name.lower())

def build_row_widget(path: Path):
    thumb = make_thumb_bytes(path)
    img_w = widgets.Image(value=thumb, format="png", width=THUMB_DISPLAY_WIDTH) if thumb else widgets.Label("No preview")

    name_label = widgets.HTML(f"<b>{path.name}</b>")
    new_name = widgets.Text(value=path.stem, layout=widgets.Layout(width="280px"))
    ext_label = widgets.Label(path.suffix)
    msg = widgets.HTML("")
    rename_btn = widgets.Button(description="Rename", tooltip="이 파일만 이름 변경")
    zoom_btn = widgets.Button(description="Zoom", tooltip="크게 보기")
    zoom_out = widgets.Output()
    row = {
        "widget": None,
        "path": path,                # 항상 최신 경로 유지
        "new_name_widget": new_name,
        "msg_widget": msg,
        "thumb_widget": img_w,
        "name_label": name_label,
        "ext": path.suffix,
        "zoom_out": zoom_out,
        "zoom_open": False
    }

    def do_rename(_=None):
        p = row["path"]
        parent, ext = p.parent, row["ext"]
        stem = row["new_name_widget"].value.strip()
        ok, err = validate_windows_filename(stem)
        if not ok:
            row["msg_widget"].value = f"<span style='color:#c00'>❌ {err}</span>"; return
        target = parent / (stem + ext)
        try:
            if target == p:
                row["msg_widget"].value = "<span style='color:#777'>변경 없음</span>"; return
            if target.exists() and not overwrite_chk.value:
                row["msg_widget"].value = "<span style='color:#c00'>❌ 동일한 이름이 이미 존재합니다.</span>"; return
            shutil.move(str(p), str(target))
            row["path"] = target
            row["name_label"].value = f"<b>{target.name}</b>"
            new_thumb = make_thumb_bytes(target)
            if new_thumb: row["thumb_widget"].value = new_thumb
            row["msg_widget"].value = "<span style='color:#070'>✅ Renamed</span>"
        except Exception as e:
            row["msg_widget"].value = f"<span style='color:#c00'>❌ {e}</span>"

    def do_zoom(_=None):
        row["zoom_open"] = not row["zoom_open"]
        row["zoom_out"].clear_output()
        if row["zoom_open"]:
            zb = make_zoom_bytes(row["path"])
            with row["zoom_out"]:
                if zb: display(widgets.Image(value=zb, format="png", width=800))
                else:  display(widgets.HTML("<i>미리보기를 생성할 수 없습니다.</i>"))

    rename_btn.on_click(do_rename)
    zoom_btn.on_click(do_zoom)

    row["widget"] = widgets.VBox([
        widgets.HBox([
            img_w,
            widgets.VBox([
                name_label,
                widgets.HBox([widgets.Label("New:"), new_name, ext_label, rename_btn, zoom_btn]),
                msg,
            ], layout=widgets.Layout(align_items="flex-start", width="100%"))
        ], layout=widgets.Layout(border="1px solid #eee", padding="6px", margin="3px 0")),
        zoom_out
    ])
    return row

def render_page(page:int):
    """현재 페이지(50개)만 위젯 생성해서 표시"""
    global _curr_page, _file_rows
    _curr_page = page
    start = (page-1)*PAGE_SIZE
    end = start + PAGE_SIZE
    files = _all_files[start:end]
    _file_rows = [build_row_widget(p) for p in files]
    items_box.children = [r["widget"] for r in _file_rows]

    # 위/아래 동일한 Pager 구성
    def _build_pager():
        prev_btn = widgets.Button(description="<< Prev 50", disabled=(page<=1))
        page_lbl = widgets.Label(f"Page {page}/{_total_pages}   (Showing {start+1}-{min(end, len(_all_files))} / {len(_all_files)})")
        next_btn = widgets.Button(description="Next 50 >>", disabled=(page>=_total_pages))

        def go_prev(_):
            if _curr_page > 1:
                render_page(_curr_page-1)
        def go_next(_):
            if _curr_page < _total_pages:
                render_page(_curr_page+1)

        prev_btn.on_click(go_prev); next_btn.on_click(go_next)
        return widgets.HBox([prev_btn, page_lbl, next_btn])

    pager_box_top.children = (_build_pager(),)
    pager_box_bottom.children = (_build_pager(),)

def on_load(_=None):
    folder = Path(dir_text.value)
    status_out.clear_output()
    with status_out:
        if not folder.exists() or not folder.is_dir():
            print(f"❌ 폴더가 존재하지 않습니다: {folder}"); return
        print(f"📂 로딩 중: {folder}")
    global _all_files, _total_pages
    _all_files = list_images(folder)
    _total_pages = max(1, (len(_all_files)+PAGE_SIZE-1)//PAGE_SIZE)
    render_page(1)
    with status_out:
        print(f"✅ 이미지 {len(_all_files)}개 로드 완료. (페이지당 {PAGE_SIZE}개)")

def on_rename_all(_=None):
    # 현재 페이지(표시 중인 50개만) 일괄 리네임
    # 1) 유효성 사전검증
    for row in _file_rows:
        ok, err = validate_windows_filename(row["new_name_widget"].value)
        if not ok:
            row["msg_widget"].value = f"<span style='color:#c00'>❌ {err}</span>"
            return
    # 2) 실행
    changed = errors = 0
    for row in _file_rows:
        p = row["path"]
        target = p.with_name(row["new_name_widget"].value.strip() + row["ext"])
        try:
            if target == p:
                row["msg_widget"].value = "<span style='color:#777'>변경 없음</span>"
                continue
            if target.exists() and not overwrite_chk.value:
                row["msg_widget"].value = "<span style='color:#c00'>❌ 동일한 이름이 이미 존재합니다.</span>"
                errors += 1; continue
            shutil.move(str(p), str(target))
            row["path"] = target
            row["name_label"].value = f"<b>{target.name}</b>"
            new_thumb = make_thumb_bytes(target)
            if new_thumb: row["thumb_widget"].value = new_thumb
            row["msg_widget"].value = "<span style='color:#070'>✅ Renamed</span>"
            changed += 1
        except Exception as e:
            row["msg_widget"].value = f"<span style='color:#c00'>❌ {e}</span>"
            errors += 1
    with status_out:
        print(f"완료: 변경 {changed}건, 오류 {errors}건")

load_btn.on_click(on_load)
rename_all_btn.on_click(on_rename_all)

# 최초 로드
on_load()


HBox()

VBox()

HBox()

Output(layout=Layout(border_bottom='1px solid #ddd', border_left='1px solid #ddd', border_right='1px solid #dd…